<a href="https://colab.research.google.com/github/dnevo/Practicum---project-6/blob/master/Practicum_project_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# following lines are specific to Google Colab
#%load_ext google.colab.data_table
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Practicum Yandex - Project 6'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Practicum Yandex - Project 6


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:11,.2f}'.format

In [0]:
df = pd.read_csv('datasets/users_behavior.csv',index_col=0)
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,"19,915.42",0
1,85.00,516.75,56.00,"22,696.96",0
2,77.00,467.66,86.00,"21,060.45",0
3,106.00,745.53,81.00,"8,437.39",1
4,66.00,418.74,1.00,"14,502.75",0


The dataset is splitted into 3 groups according to the common practice: train 60%, Validation 20%, test 20%.

Since the train_test_split method is designed to split into only 2 groups, we are calling it twice in order to split to three.

In [0]:
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=12345)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Random Forest: best result with n_estimator = 18 (0.793)

In [0]:
for n_estim in range(1,20):
    model = RandomForestClassifier(random_state=12345, n_estimators=n_estim)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("RandomForest n_estimator =", n_estim, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

RandomForest n_estimator = 1 : 0.7107309486780715
RandomForest n_estimator = 2 : 0.7636080870917574
RandomForest n_estimator = 3 : 0.7387247278382582
RandomForest n_estimator = 4 : 0.7713841368584758
RandomForest n_estimator = 5 : 0.749611197511664
RandomForest n_estimator = 6 : 0.7807153965785381
RandomForest n_estimator = 7 : 0.7682737169517885
RandomForest n_estimator = 8 : 0.7822706065318819
RandomForest n_estimator = 9 : 0.7729393468118196
RandomForest n_estimator = 10 : 0.7853810264385692
RandomForest n_estimator = 11 : 0.7838258164852255
RandomForest n_estimator = 12 : 0.7869362363919129
RandomForest n_estimator = 13 : 0.7822706065318819
RandomForest n_estimator = 14 : 0.7838258164852255
RandomForest n_estimator = 15 : 0.7838258164852255
RandomForest n_estimator = 16 : 0.7869362363919129
RandomForest n_estimator = 17 : 0.7869362363919129
RandomForest n_estimator = 18 : 0.7931570762052877
RandomForest n_estimator = 19 : 0.7884914463452566


Logistic Regression: accuracy on validation set: 0.711

In [0]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print("LogisticRegression:", accuracy_score(target_valid, predictions_valid))

LogisticRegression: 0.7107309486780715


Decision Tree: best result is with max_depth = 3 (0.785)

In [0]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("DecisionTree max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

DecisionTree max_depth = 1 : 0.7542768273716952
DecisionTree max_depth = 2 : 0.7822706065318819
DecisionTree max_depth = 3 : 0.7853810264385692
DecisionTree max_depth = 4 : 0.7791601866251944
DecisionTree max_depth = 5 : 0.7791601866251944


Summary of above: we have made a comparison of 3 models: Random Forest, Logistic Regression and Decision Tree. As from the results, Random Forest (with accuracy _estimator = 18) provides best accuracy: 79.3%

In [0]:
model = RandomForestClassifier(random_state=12345, n_estimators=18)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7931570762052877


The last step, is checking accuracy on the test set. The accuracy remains similar 78.5% - which is above the target of 75% that we have set.

In [0]:
predictions_test = model.predict(features_test)
print(accuracy_score(target_test, predictions_test))

0.7853810264385692


In [0]:
joblib.dump(model, 'model.joblib')
model = joblib.load('model.joblib')